# Sentence embeddings

In [26]:
import sentence_transformers
import numpy as np
from tqdm import tqdm_notebook

In [27]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

## Clean data and comments
- For data:
    - read id, title, selftext, and score
    - remove [removed] and [deleted], drop NAs, and strip white spaces.
- For comments:
    - read data and remove strings in score column
    - read parent_id, body, and score
    - remove [removed] and [deleted], drop NAs, and strip white spaces.
    - filter out t1_ comments. These are indirect comments.
    - keep t3_ id, and remain body and score.

In [5]:
import pandas as pd

def clean_data(csv_in):
    csv_read = pd.read_csv(csv_in)

    csv = csv_read[['id', 'title', 'selftext', 'score']]
    csv = csv[csv.selftext != '[removed]']
    csv = csv[csv.selftext != '[deleted]']
    csv.dropna(subset = ["selftext"], inplace=True)
    csv = csv.replace(r'^\s+','', regex=True)
    return csv

nosleep2020 = clean_data('../Creepy Data/NoSleep/RS_2020_nosleep.csv')

confessions2020 = clean_data('../Creepy Data/Confessions/RS_2020_confessions.csv')

In [6]:
def clean_comments(csv_in):
    csv_read = pd.read_csv(csv_in, dtype = str)
    csv = csv_read[csv_read['score'].apply(pd.to_numeric, errors='coerce').notna()]
    csv = csv[['parent_id', 'body', 'score']]
    csv = csv[csv.body != '[removed]']
    csv = csv[csv.body != '[deleted]']
    csv.dropna(subset = ["body"], inplace=True)
    csv = csv.replace(r'^\s+','', regex=True)
    
    # t3_ are direct comments
    csv = csv[csv.parent_id.str.startswith('t3_')]
    parent_id = csv.parent_id.str.split(r"_", expand=True)
    
    result = pd.concat([csv, parent_id], axis=1)
    result = result[[1, 'body', 'score']]
    result.columns = ['parent_id', 'body', 'score']
    return result
#     return csv
    
nosleep_comments = clean_comments('../Creepy Data/NoSleep/nosleep_comments.csv')
confessions_comments = clean_comments('../Creepy Data/Confessions/confessions_comments.csv')

## Pickle dfs for later use

In [46]:
nosleep2020.to_pickle('../pickles/nosleep2020.pickle')
confessions2020.to_pickle('../pickles/confessions2020.pickle')
nosleep_comments.to_pickle('../pickles/nosleep_comments.pickle')
confessions_comments.to_pickle('../pickles/confessions_comments.pickle')

In [66]:
import pandas as pd
nosleep2020 = pd.read_pickle('../pickles/nosleep2020.pickle')
confessions2020 = pd.read_pickle('../pickles/confessions2020.pickle')
nosleep_comments = pd.read_pickle('../pickles/nosleep_comments.pickle')
confessions_comments = pd.read_pickle('../pickles/confessions_comments.pickle')

nosleep2020 = nosleep2020[:100].copy()
confessions2020 = confessions2020[:100].copy()
nosleep_comments = nosleep_comments[:10000].copy()
confessions_comments = confessions_comments[:10000].copy()

In [88]:
pd.set_option('display.max_colwidth', 20)
nosleep_comments.merge(nosleep2020, left_on='parent_id', right_on='id').groupby(['title', 'selftext', 'score_y', 'body']).first().loc[:, ['score_x']]


score_x
title                selftext             score_y body                        
(Trigger warning)... I don't know if t... 1       Hi there! [Your p...       1
A man knocked on ... “Hi, uh, I’m Rob ... 1       Aww poor dogs. Al...       5
                                                  I could only imag...       3
                                                  I have it when an...       2
                                                  I hope you will u...       1
...                                                                        ...
When I was younge... [Part 1 ](https:/... 1       It looks like the...       1
                                                  SMACK THE BODY WI...       4
You always loved ... You always loved ... 1       wow... thought it...       2
just not sleeping... drunk guy gets ha... 1       Your submission h...       1
sleep is the leas... drunk guy gets ha... 1       Your submission h...       1

[270 rows x 1 columns]

## Using spaCy to separate sentences and sentence_transformers to get sentence vectors.

## Find linear combinations of r/NoSleep 's comments, grouped by parent_id

In [30]:
import spacy

spacy.prefer_gpu()
# Must return True for GPU to work

True

In [31]:
nlp = spacy.load('en_core_web_lg')

In [32]:
# need nlp and model
def sum_vec(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    comment_vec = model.encode(sentences)
    return np.sum(comment_vec, axis = 0)

In [33]:
from tqdm import tqdm, tqdm_pandas, tqdm_notebook
tqdm.pandas()

/home/anthony/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [38]:
nosleep_comments.loc[:,'sum_vec'] = nosleep_comments.progress_apply(lambda x: sum_vec(x['body']), axis=1)

nosleep_comments.loc[:,'score'] = pd.to_numeric(nosleep_comments.loc[:,'score'])

# print(nosleep_comments_short.score.dtypes)
nosleep_comments['lc'] = nosleep_comments['score'].to_numpy() * nosleep_comments['sum_vec'].to_numpy()
nosleep_comments.groupby(['parent_id'])['lc'].apply(np.sum)

100%|██████████| 10000/10000 [02:57<00:00, 56.36it/s]


parent_id
e2s3p5    [2.1981773, 0.5333843, 2.0460103, -0.97413677,...
e334ka    [0.32944906, -0.72205037, -0.4612019, -0.18653...
e48fc1    [0.2841142, -0.24578956, 0.03323673, 0.1975626...
e5732j    [-0.18932578, 0.064817116, -0.5032157, -0.1876...
e6p539    [-0.35783115, 0.22775733, -0.50352436, -1.2612...
                                ...                        
gxrnj7    [-0.42676052, -0.80316484, 0.20431642, -0.4659...
gxrudc    [-1.0927082, 0.8121819, 0.30228412, -0.0302171...
gxs69h    [-1.0927082, 0.8121819, 0.30228412, -0.0302171...
gxs6jf    [-1.245924, 5.5134716, 2.124174, -4.7967, -1.8...
gxsa0i    [-4.0768266, 1.1921716, 1.8977762, -2.6812117,...
Name: lc, Length: 2696, dtype: object

In [39]:
nosleep_comments

,parent_id,body,score,sum_vec,lc
4,gxq0yh,This is truly fascinating!! \n\nConsider makin...,1,"[0.58918774, -0.41969723, 0.74375904, -0.36868...","[0.58918774, -0.41969723, 0.74375904, -0.36868..."
5,gxg5z1,just.... no,2,"[-0.87192535, -0.15856919, -0.4702785, 0.14323...","[-1.7438507, -0.31713837, -0.940557, 0.2864617..."
6,gxjuvy,Im tearing up oh God youre such a lovely soul,2,"[-0.7478468, 0.63004714, 0.34330124, -0.496975...","[-1.4956936, 1.2600943, 0.6866025, -0.99395, -..."
8,gp0040,Okay - had to pause in my reading for a moment...,1,"[-0.06602657, 0.3970722, 0.1914272, -0.9652371...","[-0.06602657, 0.3970722, 0.1914272, -0.9652371..."
9,gxieer,I thought it was a cheerful story till the las...,2,"[-0.051630713, -0.75132173, 1.4272715, -0.7462...","[-0.103261426, -1.5026435, 2.854543, -1.492407..."
...,...,...,...,...,...
19314,gm96vn,Your submission has been removed. Stories in /...,1,"[-1.0927082, 0.8121819, 0.30228412, -0.0302171...","[-1.0927082, 0.8121819, 0.30228412, -0.0302171..."
19315,gm7aks,"Well, why not put a ring of salt around it fir...",4,"[0.7479411, 0.53833973, 0.06780348, 0.37441745...","[2.9917643, 2.153359, 0.27121392, 1.4976698, -..."
19318,gm90lz,It looks like there may be more to this story....,1,"[-3.8055224, 2.069212, 2.6158676, -3.3476005, ...","[-3.8055224, 2.069212, 2.6158676, -3.3476005, ..."
19322,glfrda,That's so sad...,1,"[0.15047048, 0.35812125, -0.27485484, -0.31914...","[0.15047048, 0.35812125, -0.27485484, -0.31914..."


In [ ]:
nosleep_comments.to_pickle('../pickles/nosleep_comments.pickle')

### Find sum_vec of r/NoSleep

In [40]:
nosleep2020

,id,title,selftext,score
0,gxsa0i,Do NOT Open Your Eyes... (Pt. 1),This is the only rule of our household. If you...,1
1,gxs6jf,Do NOT open your eyes. (The Beginning),This is the only rule of our household. If you...,1
3,gxrytp,My Best Friend Saw Bugs Under His Skin,It is hard for me to talk about my old friend ...,1
5,gxrnj7,"I picked up a hitchhiker by mistake, now he's ...",They say the devil is in the details. Well th...,1
6,gxrm8v,I'm tasked with killing nameless things out in...,“Any sign of ‘em yet?” \n\nI continued staring...,1
...,...,...,...,...
124,gxao8c,The old house in the valley,The following events happened during the early...,1
125,gxanrv,"I went to my friend’s sixteenth birthday, and ...","Do you ever get that feeling…?\n\nYou know, th...",1
126,gxan8l,"It's not like I wanted to, you know.","""Dad, are we poor?""\n\nThat's a hot knife to t...",1
130,gxa2c9,Help me find someone. It's important.,Todd meandered his way all over town. He was o...,1


In [41]:
nosleep2020.loc[:,'sum_vec'] = nosleep2020.progress_apply(lambda x: sum_vec(x['selftext']), axis=1)

100%|██████████| 100/100 [00:16<00:00,  6.06it/s]


In [42]:
nosleep2020

,id,title,selftext,score,sum_vec
0,gxsa0i,Do NOT Open Your Eyes... (Pt. 1),This is the only rule of our household. If you...,1,"[2.940986, -5.617336, 11.896912, -15.436118, -..."
1,gxs6jf,Do NOT open your eyes. (The Beginning),This is the only rule of our household. If you...,1,"[2.940986, -5.617336, 11.896912, -15.436118, -..."
3,gxrytp,My Best Friend Saw Bugs Under His Skin,It is hard for me to talk about my old friend ...,1,"[-19.201273, -13.715499, 24.393753, -33.97739,..."
5,gxrnj7,"I picked up a hitchhiker by mistake, now he's ...",They say the devil is in the details. Well th...,1,"[-60.77887, -33.726135, 119.47121, -95.021385,..."
6,gxrm8v,I'm tasked with killing nameless things out in...,“Any sign of ‘em yet?” \n\nI continued staring...,1,"[-22.785084, 14.806147, 26.129469, -24.832222,..."
...,...,...,...,...,...
124,gxao8c,The old house in the valley,The following events happened during the early...,1,"[-81.15556, 9.352104, 10.277876, -108.00954, 9..."
125,gxanrv,"I went to my friend’s sixteenth birthday, and ...","Do you ever get that feeling…?\n\nYou know, th...",1,"[-22.736895, 5.065634, 9.346256, -30.13725, 0...."
126,gxan8l,"It's not like I wanted to, you know.","""Dad, are we poor?""\n\nThat's a hot knife to t...",1,"[-67.79279, 12.359209, 21.70082, -84.93899, 13..."
130,gxa2c9,Help me find someone. It's important.,Todd meandered his way all over town. He was o...,1,"[-33.275112, -34.9166, -4.4392314, -27.243034,..."


In [ ]:
nosleep2020.to_pickle('../pickles/nosleep2020.pickle')

In [43]:
nosleep2020

,id,title,selftext,score,sum_vec
0,gxsa0i,Do NOT Open Your Eyes... (Pt. 1),This is the only rule of our household. If you...,1,"[2.940986, -5.617336, 11.896912, -15.436118, -..."
1,gxs6jf,Do NOT open your eyes. (The Beginning),This is the only rule of our household. If you...,1,"[2.940986, -5.617336, 11.896912, -15.436118, -..."
3,gxrytp,My Best Friend Saw Bugs Under His Skin,It is hard for me to talk about my old friend ...,1,"[-19.201273, -13.715499, 24.393753, -33.97739,..."
5,gxrnj7,"I picked up a hitchhiker by mistake, now he's ...",They say the devil is in the details. Well th...,1,"[-60.77887, -33.726135, 119.47121, -95.021385,..."
6,gxrm8v,I'm tasked with killing nameless things out in...,“Any sign of ‘em yet?” \n\nI continued staring...,1,"[-22.785084, 14.806147, 26.129469, -24.832222,..."
...,...,...,...,...,...
124,gxao8c,The old house in the valley,The following events happened during the early...,1,"[-81.15556, 9.352104, 10.277876, -108.00954, 9..."
125,gxanrv,"I went to my friend’s sixteenth birthday, and ...","Do you ever get that feeling…?\n\nYou know, th...",1,"[-22.736895, 5.065634, 9.346256, -30.13725, 0...."
126,gxan8l,"It's not like I wanted to, you know.","""Dad, are we poor?""\n\nThat's a hot knife to t...",1,"[-67.79279, 12.359209, 21.70082, -84.93899, 13..."
130,gxa2c9,Help me find someone. It's important.,Todd meandered his way all over town. He was o...,1,"[-33.275112, -34.9166, -4.4392314, -27.243034,..."


In [44]:
nosleep_merged = nosleep_comments.merge(nosleep2020, left_on='parent_id', right_on='id')

In [45]:
nosleep_merged

,parent_id,body,score_x,sum_vec_x,lc,id,title,selftext,score_y,sum_vec_y
0,gxq0yh,This is truly fascinating!! \n\nConsider makin...,1,"[0.58918774, -0.41969723, 0.74375904, -0.36868...","[0.58918774, -0.41969723, 0.74375904, -0.36868...",gxq0yh,"That which kills me, makes me stronger.",I often dream about Tsunamis.\n\nIt's always t...,1,"[-5.738542, 0.6425567, 81.09605, -53.417187, -..."
1,gxq0yh,Woah...just woah. I got questions but too spee...,1,"[0.20346296, -0.94261086, 0.28327927, 0.676400...","[0.20346296, -0.94261086, 0.28327927, 0.676400...",gxq0yh,"That which kills me, makes me stronger.",I often dream about Tsunamis.\n\nIt's always t...,1,"[-5.738542, 0.6425567, 81.09605, -53.417187, -..."
2,gxq0yh,If you or someone you know is contemplating su...,1,"[0.68557304, -0.67809576, 1.6547089, -3.315929...","[0.68557304, -0.67809576, 1.6547089, -3.315929...",gxq0yh,"That which kills me, makes me stronger.",I often dream about Tsunamis.\n\nIt's always t...,1,"[-5.738542, 0.6425567, 81.09605, -53.417187, -..."
3,gxg5z1,just.... no,2,"[-0.87192535, -0.15856919, -0.4702785, 0.14323...","[-1.7438507, -0.31713837, -0.940557, 0.2864617...",gxg5z1,A man knocked on my door and offered me 10% of...,"“Hi, uh, I’m Rob and I was wondering if you ar...",1,"[-10.211775, 5.102462, 7.233557, -28.880627, 0..."
4,gxg5z1,I hope you will update us on your situation!!!...,1,"[-1.2264739, 0.58753633, 0.33913088, -0.622941...","[-1.2264739, 0.58753633, 0.33913088, -0.622941...",gxg5z1,A man knocked on my door and offered me 10% of...,"“Hi, uh, I’m Rob and I was wondering if you ar...",1,"[-10.211775, 5.102462, 7.233557, -28.880627, 0..."
...,...,...,...,...,...,...,...,...,...,...
265,gxc6re,It looks like there may be more to this story....,1,"[-3.794889, 2.0691934, 2.2411857, -2.2404509, ...","[-3.794889, 2.0691934, 2.2411857, -2.2404509, ...",gxc6re,I went camping with my wife and I finally left...,[Part 1](https://www.reddit.com/r/nosleep/comm...,1,"[10.093164, -8.551209, 7.219606, -39.01724, -4..."
266,gxb2a9,It kinda sucks there was nothing down there th...,1,"[-0.33620068, -0.5459096, 0.2572607, 0.1981732...","[-0.33620068, -0.5459096, 0.2572607, 0.1981732...",gxb2a9,The house in the valley,The following events happened during the early...,1,"[-81.15556, 9.352104, 10.277876, -108.00954, 9..."
267,gxb4r5,could you atleast send the story over directs?...,1,"[0.1753691, 0.18572092, 0.23589021, -0.1620083...","[0.1753691, 0.18572092, 0.23589021, -0.1620083...",gxb4r5,The Shadow,It was a Monday. A day which was hated by many...,1,"[-12.212687, -14.0347185, 31.983816, -60.9217,..."
268,gxb4r5,bruh i was reading it and actually curious to ...,1,"[-0.01330058, 0.13857275, 0.5788143, -0.082007...","[-0.01330058, 0.13857275, 0.5788143, -0.082007...",gxb4r5,The Shadow,It was a Monday. A day which was hated by many...,1,"[-12.212687, -14.0347185, 31.983816, -60.9217,..."


In [46]:
nosleep_merged.columns = ['parent_id', 'comment', 'c_score', 'c_sum_vec', 'c_lc', 'id', 'title', 'selftext', 't_score', 't_sum_vec']

In [47]:
len(nosleep_merged.id.unique())

73

# Groupby each post then by comments

In [51]:
# pd.set_option('display.max_rows', 50)
# nosleep_merged.loc[nosleep_merged.id == 'gxieer', :].groupby(['title', 'comment']).first()

In [48]:

comment_lc = pd.DataFrame(nosleep_merged.groupby(['id'])['c_lc'].apply(np.sum))
# comment_lc.reset_index(inplace=True) # Resets the index, makes factor a column
# comment_lc.drop("comment",axis=1,inplace=True) # drop factor from axis 1 and make changes permanent by inplace=True
comment_lc

,c_lc
id,
gx9z4v,"[0.8884156, -0.82006365, 4.958798, -0.28936863..."
gxa2c9,"[-3.6786094, 1.8539501, 1.8513423, -2.2749062,..."
gxan8l,"[-19.837536, -5.8152175, 9.576509, -34.185574,..."
gxazgc,"[-3.6895905, 2.3371148, 17.484577, -14.772115,..."
gxb2a9,"[-0.33620068, -0.5459096, 0.2572607, 0.1981732..."
...,...
gxr4ff,"[-3.4779508, 1.8638754, 3.4196403, -3.8453462,..."
gxr7hh,"[-3.3774014, -0.084626645, 3.8341122, -4.97267..."
gxrnj7,"[-0.42676052, -0.80316484, 0.20431642, -0.4659..."


In [58]:
nosleep_merged_gb_id = nosleep_merged.groupby(['id']).first()

In [50]:
# print(nosleep_comments_short.score.dtypes)
nosleep_merged_gb_id['t_lc'] = nosleep_merged_gb_id['t_score'].to_numpy() * nosleep_merged_gb_id['t_sum_vec'].to_numpy()
nosleep_merged_gb_id.groupby(['parent_id'])['t_lc'].apply(np.sum)

parent_id
gx9z4v    [-33.031883, -108.33711, 49.864426, -118.04858...
gxa2c9    [-33.275112, -34.9166, -4.4392314, -27.243034,...
gxan8l    [-67.79279, 12.359209, 21.70082, -84.93899, 13...
gxazgc    [-37.972485, 7.2398953, 59.625057, -101.77551,...
gxb2a9    [-81.15556, 9.352104, 10.277876, -108.00954, 9...
                                ...                        
gxr4ff    [-5.7507873, -26.91448, 14.6927805, -42.703568...
gxr7hh    [-3.1201575, 0.2122544, 12.899367, -13.549828,...
gxrnj7    [-60.77887, -33.726135, 119.47121, -95.021385,...
gxs6jf    [2.940986, -5.617336, 11.896912, -15.436118, -...
gxsa0i    [2.940986, -5.617336, 11.896912, -15.436118, -...
Name: t_lc, Length: 73, dtype: object

In [51]:
nosleep_merged_gb_id

,parent_id,comment,c_score,c_sum_vec,c_lc,title,selftext,t_score,t_sum_vec,t_lc
id,,,,,,,,,,
gx9z4v,gx9z4v,I guess the doctors have already found the neu...,2,"[-0.25842425, 0.07537876, 0.7656959, 0.1256587...","[-0.5168485, 0.15075752, 1.5313919, 0.25131747...",My wife was acting weird and I finally found o...,My wife was acting weird and I finally found o...,2,"[-16.515942, -54.168556, 24.932213, -59.024292...","[-33.031883, -108.33711, 49.864426, -118.04858..."
gxa2c9,gxa2c9,It looks like there may be more to this story....,1,"[-3.6786094, 1.8539501, 1.8513423, -2.2749062,...","[-3.6786094, 1.8539501, 1.8513423, -2.2749062,...",Help me find someone. It's important.,Todd meandered his way all over town. He was o...,1,"[-33.275112, -34.9166, -4.4392314, -27.243034,...","[-33.275112, -34.9166, -4.4392314, -27.243034,..."
gxan8l,gxan8l,I don't blame you for going to whatever length...,1,"[0.3690974, -0.4249894, 0.22890645, -1.1899724...","[0.3690974, -0.4249894, 0.22890645, -1.1899724...","It's not like I wanted to, you know.","""Dad, are we poor?""\n\nThat's a hot knife to t...",1,"[-67.79279, 12.359209, 21.70082, -84.93899, 13...","[-67.79279, 12.359209, 21.70082, -84.93899, 13..."
gxazgc,gxazgc,I think the story was great keep up the great ...,1,"[0.68025696, -0.34426588, 0.5134946, -0.666616...","[0.68025696, -0.34426588, 0.5134946, -0.666616...",Do not visit the train station past midnight,I'm in my last year of high school and live in...,1,"[-37.972485, 7.2398953, 59.625057, -101.77551,...","[-37.972485, 7.2398953, 59.625057, -101.77551,..."
gxb2a9,gxb2a9,It kinda sucks there was nothing down there th...,1,"[-0.33620068, -0.5459096, 0.2572607, 0.1981732...","[-0.33620068, -0.5459096, 0.2572607, 0.1981732...",The house in the valley,The following events happened during the early...,1,"[-81.15556, 9.352104, 10.277876, -108.00954, 9...","[-81.15556, 9.352104, 10.277876, -108.00954, 9..."
...,...,...,...,...,...,...,...,...,...,...
gxr4ff,gxr4ff,"Well written, and an interesting story. Lookin...",1,"[0.1701939, -0.21244906, 1.1521204, -1.5114393...","[0.1701939, -0.21244906, 1.1521204, -1.5114393...",No one in my family is ever buried or cremated...,"It's a strict rule in my family, usually adher...",1,"[-5.7507873, -26.91448, 14.6927805, -42.703568...","[-5.7507873, -26.91448, 14.6927805, -42.703568..."
gxr7hh,gxr7hh,"I know what you feel, not because I am in your...",1,"[-3.3774014, -0.084626645, 3.8341122, -4.97267...","[-3.3774014, -0.084626645, 3.8341122, -4.97267...",General Rantish,So basically i have just been feeling very dow...,1,"[-3.1201575, 0.2122544, 12.899367, -13.549828,...","[-3.1201575, 0.2122544, 12.899367, -13.549828,..."
gxrnj7,gxrnj7,...and just the revolution of an engine..,1,"[-0.42676052, -0.80316484, 0.20431642, -0.4659...","[-0.42676052, -0.80316484, 0.20431642, -0.4659...","I picked up a hitchhiker by mistake, now he's ...",They say the devil is in the details. Well th...,1,"[-60.77887, -33.726135, 119.47121, -95.021385,...","[-60.77887, -33.726135, 119.47121, -95.021385,..."


In [52]:
text_lc = nosleep_merged_gb_id.loc[:,['t_lc']]
text_lc

,t_lc
id,
gx9z4v,"[-33.031883, -108.33711, 49.864426, -118.04858..."
gxa2c9,"[-33.275112, -34.9166, -4.4392314, -27.243034,..."
gxan8l,"[-67.79279, 12.359209, 21.70082, -84.93899, 13..."
gxazgc,"[-37.972485, 7.2398953, 59.625057, -101.77551,..."
gxb2a9,"[-81.15556, 9.352104, 10.277876, -108.00954, 9..."
...,...
gxr4ff,"[-5.7507873, -26.91448, 14.6927805, -42.703568..."
gxr7hh,"[-3.1201575, 0.2122544, 12.899367, -13.549828,..."
gxrnj7,"[-60.77887, -33.726135, 119.47121, -95.021385,..."


In [53]:
nosleep = pd.merge(text_lc, comment_lc, on='id', how = 'outer')

In [54]:
nosleep

,t_lc,c_lc
id,,
gx9z4v,"[-33.031883, -108.33711, 49.864426, -118.04858...","[0.8884156, -0.82006365, 4.958798, -0.28936863..."
gxa2c9,"[-33.275112, -34.9166, -4.4392314, -27.243034,...","[-3.6786094, 1.8539501, 1.8513423, -2.2749062,..."
gxan8l,"[-67.79279, 12.359209, 21.70082, -84.93899, 13...","[-19.837536, -5.8152175, 9.576509, -34.185574,..."
gxazgc,"[-37.972485, 7.2398953, 59.625057, -101.77551,...","[-3.6895905, 2.3371148, 17.484577, -14.772115,..."
gxb2a9,"[-81.15556, 9.352104, 10.277876, -108.00954, 9...","[-0.33620068, -0.5459096, 0.2572607, 0.1981732..."
...,...,...
gxr4ff,"[-5.7507873, -26.91448, 14.6927805, -42.703568...","[-3.4779508, 1.8638754, 3.4196403, -3.8453462,..."
gxr7hh,"[-3.1201575, 0.2122544, 12.899367, -13.549828,...","[-3.3774014, -0.084626645, 3.8341122, -4.97267..."
gxrnj7,"[-60.77887, -33.726135, 119.47121, -95.021385,...","[-0.42676052, -0.80316484, 0.20431642, -0.4659..."


In [55]:
nosleep['output'] = nosleep.apply(np.sum, axis = 1)
nosleep['creepy'] = 1

In [56]:
nosleep

,t_lc,c_lc,output,creepy
id,,,,
gx9z4v,"[-33.031883, -108.33711, 49.864426, -118.04858...","[0.8884156, -0.82006365, 4.958798, -0.28936863...","[-32.143467, -109.15717, 54.823223, -118.33795...",1
gxa2c9,"[-33.275112, -34.9166, -4.4392314, -27.243034,...","[-3.6786094, 1.8539501, 1.8513423, -2.2749062,...","[-36.95372, -33.06265, -2.5878892, -29.51794, ...",1
gxan8l,"[-67.79279, 12.359209, 21.70082, -84.93899, 13...","[-19.837536, -5.8152175, 9.576509, -34.185574,...","[-87.630325, 6.5439916, 31.27733, -119.12456, ...",1
gxazgc,"[-37.972485, 7.2398953, 59.625057, -101.77551,...","[-3.6895905, 2.3371148, 17.484577, -14.772115,...","[-41.662075, 9.57701, 77.109634, -116.54763, 3...",1
gxb2a9,"[-81.15556, 9.352104, 10.277876, -108.00954, 9...","[-0.33620068, -0.5459096, 0.2572607, 0.1981732...","[-81.49176, 8.806194, 10.535136, -107.81136, 9...",1
...,...,...,...,...
gxr4ff,"[-5.7507873, -26.91448, 14.6927805, -42.703568...","[-3.4779508, 1.8638754, 3.4196403, -3.8453462,...","[-9.228738, -25.050606, 18.112421, -46.548912,...",1
gxr7hh,"[-3.1201575, 0.2122544, 12.899367, -13.549828,...","[-3.3774014, -0.084626645, 3.8341122, -4.97267...","[-6.4975586, 0.12762776, 16.733479, -18.522501...",1
gxrnj7,"[-60.77887, -33.726135, 119.47121, -95.021385,...","[-0.42676052, -0.80316484, 0.20431642, -0.4659...","[-61.20563, -34.5293, 119.67552, -95.48733, -1...",1


In [63]:
nosleep_merged_gb_id.groupby(['title', 'comment']).first()

,,parent_id,c_score,c_sum_vec,c_lc,selftext,t_score,t_sum_vec
title,comment,,,,,,,
(Trigger warning) I don't know if I will die,"Hi there! [Your post](https://redd.it/gxdnbv) has been removed from /r/nosleep for violating the following rules:\n\n* **Invalid Tags**\n\n /r/nosleep has strict rules about tags in story titles:\n\n **Tags (example: [True], [real experience]) are not allowed.** The only thing in brackets **[]**, **{}** or parenthesis **()** should be a reference to which ""part"" of your series the post is. **Example**: (part 1) or [Pt2].\n\n**Since titles cannot be edited on Reddit, please repost your story with a corrected title.**\n\n\n\nAdditionally, the following issues have been detected in your post, which either violate rules or may make your post unreadable. Please correct them when re-posting your story.\n\n* **Long Paragraphs Detected**\n\n You have one or more paragraphs containing more than 350 words. Please break up your story into smaller paragraphs. You can create paragraphs by pressing `Enter` twice at the end of a line.\n\n_I am a bot, and this was automatically posted. Do not reply to me as messages will be ignored. Please [contact the moderators of this subreddit](https://www.reddit.com/message/compose?to=%2Fr%2Fnosleep) if you have any questions, concerns, or bugs to report._",gxdnbv,1,"[-0.74945915, 5.206074, 3.0300257, -5.245705, ...","[-0.74945915, 5.206074, 3.0300257, -5.245705, ...",I don't know if this is going to get on here. ...,1,"[-0.94779146, -19.951593, 15.985868, -33.50360..."
A man knocked on my door and offered me 10% off of pest control service. Now I wish I accepted his offer.,just.... no,gxg5z1,2,"[-0.87192535, -0.15856919, -0.4702785, 0.14323...","[-1.7438507, -0.31713837, -0.940557, 0.2864617...","“Hi, uh, I’m Rob and I was wondering if you ar...",1,"[-10.211775, 5.102462, 7.233557, -28.880627, 0..."
A man pays me 300 dollars a day to look after his house. I don’t think I should have taken the job. (Day 1),"Hi there! [Your post](https://redd.it/gxqlp8) has been removed from /r/nosleep for violating the following rules:\n\n* **Invalid Tags**\n\n /r/nosleep has strict rules about tags in story titles:\n\n **Tags (example: [True], [real experience]) are not allowed.** The only thing in brackets **[]**, **{}** or parenthesis **()** should be a reference to which ""part"" of your series the post is. **Example**: (part 1) or [Pt2].\n\n**Since titles cannot be edited on Reddit, please repost your story with a corrected title.**\n\n\n\n_I am a bot, and this was automatically posted. Do not reply to me as messages will be ignored. Please [contact the moderators of this subreddit](https://www.reddit.com/message/compose?to=%2Fr%2Fnosleep) if you have any questions, concerns, or bugs to report._",gxqlp8,1,"[-1.5123115, 5.416205, 2.0913079, -4.7316494, ...","[-1.5123115, 5.416205, 2.0913079, -4.7316494, ...","He didn’t give me a name, not his real one, at...",1,"[-0.20854497, 1.7412541, 21.58472, -26.046537,..."
A man pays me 300 dollars a day to look after his house. I don’t think I should have taken the job. (Part 1),if it were me i would be incredibly upset i didnt get to read the list prior. always read the fine print before agreeing to anything kids,gxqnck,4,"[0.20963815, -1.0266793, -0.48361447, -0.11716...","[0.8385526, -4.106717, -1.9344579, -0.4686625,...",A man pays me 300 dollars a day to look after ...,1,"[0.2982637, 1.6117228, 21.388351, -27.965477, ..."
Best Friends ForEVER,"Hi there! [Your post](https://redd.it/gxcpnj) has been **temporarily** removed from /r/nosleep due to the following formatting issues detected in your post:\n\n* **Long Paragraphs Detected**\n\n You have one or more paragraphs containing more than 350 words. Please break up your story into smaller paragraphs. You can create paragraphs by pressing `Enter` twice at the end of a line.\n\n**Once you have fixed your formatting issues, please [click here](https://www.reddit.com/message/compose?to=%2Fr%2Fnoslee